# MRI Reconstruction: a comparison between classical and learning approaches

In this notebook we will compare 2 techniques for reconstructing MRI images:
- the classical, wavelet-based, iterative reconstruction.
- the cascade-net [1]

[1] Schlemper, J., Caballero, J., Hajnal, J. V, Price, A., & Rueckert, D. (2018). A Deep Cascade of Convolutional Neural Networks for MR Image Reconstruction. IEEE Transactions on Medical Imaging, 37(2), 491–503. https://doi.org/10.1109/TMI.2017.2760978

In [1]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages (19.3.1)


In [2]:
!pip install python-pysap pysap-mri

     |████████████████████████████████| 163kB 11.8MB/s eta 0:00:01
     |████████████████████████████████| 6.5MB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 58.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 6.7MB/s  eta 0:00:01
     |████████████████████████████████| 450kB 28.2MB/s eta 0:00:01
  Created wheel for python-pysap: filename=python_pySAP-0.0.3-cp36-cp36m-linux_x86_64.whl size=3243157 sha256=f9b358838544e32c6d4094433983a3abbf6a452e3943a0ea2b0f44d688b41704
  Stored in directory: /volatile/home/Zaccharie/.cache/pip/wheels/ed/89/72/90779e2c5750a2de0de54b858a9fc4f3edbdab4cef835d0884
  Created wheel for pysap-mri: filename=pysap_mri-0.1.1-cp36-none-any.whl size=43603 sha256=a7a280af4bd6b01034cd2040cfa524934479473863bce38e5efafe3b9c5d4755
  Stored in directory: /volatile/home/Zaccharie/.cache/pip/wheels/01/2b/bc/fe1b738580dd02faa4a8b255f3d53c49a1e34d6994d5e74ed2
  Created wheel for pyqtgraph: filename=pyqtgraph-0.10.0-cp36-none-any.whl size=73

In [2]:
%matplotlib nbagg
from huggingface_hub import hf_hub_download
import matplotlib.pyplot as plt
from mri.numerics.fourier import FFT2
from mri.numerics.reconstruct import sparse_rec_fista
from mri.numerics.utils import generate_operators
from mri.numerics.utils import convert_mask_to_locations
import numpy as np

from fastmri_recon.helpers.utils import gen_mask, crop_center
from fastmri_recon.helpers.evaluate import psnr, ssim 
from fastmri_recon.models.cascading import cascade_net


                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.4.1     installed
numpy          : >=1.16.4  - required | 1.16.4    installed
matplotlib     : >=3.0.0   - required | 3.1.1     installed
astropy        : >=3.0.0   - required | 4.0       installed
nibabel        : >=2.3.2   - required | 2.4.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.4.1     installed
scikit-learn   : >=0.19.1  - requi

Using TensorFlow backend.


In [3]:
plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['image.cmap'] = 'gray'

In [4]:
np.random.seed(0)

# Data handling

The data comes from the fastMRI database [2]. We selected the 16th slice of the first validation file (`file1000000`).

[2] Zbontar, J., Knoll, F., Sriram, A., Muckley, M. J., Bruno, M., Defazio, A., … Lui, Y. W. (n.d.). fastMRI: An Open Dataset and Benchmarks for Accelerated MRI. Retrieved from https://arxiv.org/pdf/1811.08839.pdf

## Data loading

In [5]:
image = np.load('gt_image.npy')
kspace = np.load('gt_kspace.npy')

In [6]:
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image[..., 0])
axs[1].imshow(np.abs(kspace[..., 0]))

<IPython.core.display.Javascript object>

## Retrospective undersampling

In [7]:
AF = 4
mask = gen_mask(kspace[..., 0], accel_factor=AF, seed=0)
# the mas is received in fastmri format, we make a 1 and 0 mask in the fourier space
fourier_mask = np.repeat(mask.astype(np.float), kspace.shape[0], axis=0)
masked_kspace = fourier_mask[..., None] * kspace

In [8]:
plt.figure()
plt.imshow(fourier_mask)

<IPython.core.display.Javascript object>

In [9]:
plt.figure()
plt.imshow(crop_center(np.abs(np.fft.fftshift(np.fft.ifft2(masked_kspace[..., 0], norm='ortho'))), 320))

<IPython.core.display.Javascript object>

## Batching

Batching is necessary when using neural networks on both the training and the prediction steps.

In [10]:
mask_batch = fourier_mask[None, ...]
masked_kspace_batch = masked_kspace[None, ...]

## Scaling
Scaling was used for training to avoid numerical approximations that lead to poor training. Generally the inputs are normalized, but it was easier here to just scale them using the mean over the training set.

In [11]:
masked_kspace_batch *= 1e6

# Reconstruction using the Cascade net neural network

## Model loading

In [12]:
run_params = {
    'n_cascade': 5,
    'n_convs': 5,
    'n_filters': 48,
    'noiseless': True,
}
model_name = 'CascadeNet-fastmri'

In [13]:
model = cascade_net(input_size=(None, None, 1), fastmri=True, **run_params)
model_weights_path = hf_hub_download(
    repo_id=f'zaccharieramzi/{model_name}',
    filename='model_weights.h5',
)
model.load_weights(model_weights_path)

W0103 16:32:23.777014 139793466787584 deprecation.py:323] From /home/zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/nn_mri.py:82: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


## Reconstruction

In [14]:
%%time
cascade_reconstructed_image = model.predict_on_batch([masked_kspace_batch, mask_batch])

W0103 16:32:25.135434 139793466787584 deprecation_wrapper.py:119] From /home/zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



CPU times: user 10.4 s, sys: 474 ms, total: 10.9 s
Wall time: 4.8 s


## Visual comparison

In [15]:
fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)
axs[0].imshow(image[..., 0])
axs[1].imshow(cascade_reconstructed_image[0, ..., 0])

<IPython.core.display.Javascript object>

## Quantitative comparison

In [16]:
print('PSNR of the net reconstructed image:', psnr(image, cascade_reconstructed_image[0]/1e6))

PSNR of the net reconstructed image: 26.48027258193485


In [17]:
print('SSIM of the net reconstructed image:', ssim(image[None, ..., 0], cascade_reconstructed_image[..., 0]/1e6))

SSIM of the net reconstructed image: 0.5352749583786784


# Reconstruction using pysap

## Reformatting the data

In [18]:
kspace_squeeze = np.squeeze(kspace*1e6)
k_shape = kspace_squeeze.shape
kspace_loc = convert_mask_to_locations(fourier_mask)

## Creating the appropriate operators

In [19]:
fourier_op = FFT2(samples=kspace_loc, shape=k_shape)

/home/zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/mri/reconstruct/utils.py:74: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask[test] = 1


In [20]:
gradient_op, linear_op, prox_op, cost_op = generate_operators(
    data=kspace_squeeze,
    wavelet_name="sym8",
    samples=kspace_loc,
    nb_scales=4,
    mu=0.01,
    non_cartesian=False,
    uniform_data_shape=None,
    gradient_space="synthesis",
    padding_mode="periodization",
)

## Reconstruction
For the reconstruction we will use the FISTA algorithm

In [ ]:
max_iter = 200
x_final,  costs, metrics = sparse_rec_fista(
    gradient_op,
    linear_op,
    prox_op,
    cost_op,
    max_nb_of_iter=max_iter,
    verbose=1)
image_rec_fs = crop_center(np.abs(np.fft.fftshift(x_final)), 320)


                                                                               

N/A% (0 of 200) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


  _____             ____     _____      _
 |" ___|    ___    / __"| u |_ " _| U  /"\  u
U| |_  u   |_"_|  <\___ \/    | |    \/ _ \/
\|  _|/     | |    u___) |   /| |\   / ___ \\
 |_|      U/| |\u  |____/>> u |_|U  /_/   \_\\
 )(\\\,-.-,_|___|_,-.)(  (__)_// \\\_  \\\    >>
(__)(_/ \_)-' '-(_/(__)    (__) (__)(__)  (__)
    
 - mu:  0.01
 - lipschitz constant:  1.1000000000000685
 - data:  (640, 368)
 - wavelet:  <mri.reconstruct.linear.WaveletN object at 0x7f235c728780> - 4
 - max iterations:  200
 - image variable shape:  (640, 368)
 - alpha variable shape:  (235520,)
----------------------------------------
Starting optimization...


 97% (195 of 200) |##################### | Elapsed Time: 0:00:37 ETA:   0:00:01

## Visual comparison

In [ ]:
fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)
axs[0].imshow(image[..., 0])
axs[1].imshow(image_rec_fs)

## Quantitative comparison

In [ ]:
print('PSNR of the pysap reconstructed image:', psnr(image[..., 0], image_rec_fs/1e6))

In [ ]:
print('SSIM of the pysap reconstructed image:', ssim(image[None, ..., 0], image_rec_fs[None, ...]/1e6))